In [60]:
import pandas as pd
import numpy as np
import turicreate as tc

In [41]:
df = pd.read_excel("SpareItemDump.xlsx",engine="openpyxl")

In [42]:
df.dropna(subset = ["PurchaseOrderId"], inplace=True)

In [43]:
df.reset_index(drop=True, inplace=True)

In [44]:
df["PurchaseOrderId"]=df["PurchaseOrderId"].astype(int)

In [45]:
df

,RequestDate,RequestId,PurchaseOrderId,Vendor,Garage,Make,Model,SubCategory,Category,RequestedBrand,FulfilledBrand,Variant,ItemName,ItemQty,ItemUnitPrice,PartNbr
0,2023-07-01,303887,15791,Essarr Automotives - Bannerghatta,Canaraa Car Care Centre,Maruti Suzuki,Omni,NaN,NaN,oem,mgp,Std,Plug Wire,1.0,743.60,33700M79020
1,2023-07-01,303890,15790,Autoverse Mobility Pvt Ltd,Express Car Care,Hyundai,Eon,NaN,NaN,any,Purolator,Nill,Air Filter,5.0,164.56,PI-2272
2,2023-07-01,303893,15936,Suma Enterprises,Arodhana Synergy Private Limited,Mahindra,XUV500,NaN,NaN,oem,Mahindra,R FWD,"Bracket, Radiator Support Lower",1.0,12438.00,0102AW0400NN
3,2023-07-01,303895,15794,Essarr Automotives - Bannerghatta,Laksh,Maruti Suzuki,Swift,NaN,NaN,oem,MGP,V,Alternator Belt,2.0,382.80,95141M86JB0
4,2023-07-01,303896,15793,Essarr Automotives - Bannerghatta,Laksh,Maruti Suzuki,Swift,NaN,NaN,oem,MARUTI,V,Air Filter,5.0,171.60,13780M76M00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5356,2023-10-26,311379,18233,Grand Auto Spares,Motorlyfe Automotive,Hyundai,i20,Spark Plug,Electrical Components,oes,NGK,1.2L (83 h.p.),Spark Plug Set Of 4,1.0,180.34,LKR7E
5357,2023-10-26,311387,18239,Essarr Automotives - BG Road,V R Automotives,Maruti Suzuki,Alto 800,Door Handle,Interior and Comfort,oem,Maruti Suzuki,0.8L LX (TYPE 2) (47 h.p.),Rear Outer Door Handle - LH,1.0,237.60,82802M68K50-5PK
5358,2023-10-26,311387,18239,Essarr Automotives - BG Road,V R Automotives,Maruti Suzuki,Alto 800,Door Handle,Interior and Comfort,oem,Maruti Suzuki,0.8L LX (TYPE 2) (47 h.p.),Outer Door Handle - RH,1.0,237.60,82801M68K50-5PK
5359,2023-10-26,311388,18238,Essarr Automotives - BG Road,Wrap Auto Shop And Service Center,Maruti Suzuki,Swift,Outside Mirror Cover,Body,oem,Maruti Suzuki,1.3L LDI (TYPE 1) (74 h.p.),Mirror Cap - RH,1.0,206.80,84728M74LA0-5PK


In [46]:
d={}
for i in range(len(df)):
    if df["PurchaseOrderId"][i] in d:
        items = df["ItemName"][i].split(",")
        for item in items:
            item = item.strip()
            if item in d[df["PurchaseOrderId"][i]]:
                (d[df["PurchaseOrderId"][i]])[item] +=1
            else:
                d[df["PurchaseOrderId"][i]].update({item: 1})
    else:
        items = df["ItemName"][i].split(",")
        d[df["PurchaseOrderId"][i]]={items[0]:1}
        if len(items)>1:
            for k in range(1,len(items)):
                item = items[k].strip()
                if item in d[df["PurchaseOrderId"][i]]:
                    (d[df["PurchaseOrderId"][i]])[item] +=1
                else:
                    d[df["PurchaseOrderId"][i]].update({item: 1})

In [47]:
li=[]
for i in d:
    for j in d[i]:
        li.append([i,j,d[i][j]])
data = pd.DataFrame(li,columns=["PurchaseOrderId","product_name","purchase_count"])

In [48]:
data

,PurchaseOrderId,product_name,purchase_count
0,15791,Plug Wire,1
1,15790,Air Filter,1
2,15936,Bracket,1
3,15936,Radiator Support Lower,1
4,15794,Alternator Belt,1
...,...,...,...
5518,18233,Spark Plug Set Of 4,1
5519,18239,Rear Outer Door Handle - LH,1
5520,18239,Outer Door Handle - RH,1
5521,18238,Mirror Cap - RH,1


In [49]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [51]:
data = tc.SFrame(data)
data_dummy =  tc.SFrame(data_dummy)

In [92]:
data_dummy  = pd.read_csv("byPurchaseOrderId.csv")

In [93]:
data_dummy

,PurchaseOrderId,product_name,purchase_count,purchase_dummy
0,15791,Plug Wire,1,1
1,15790,Air Filter,1,1
2,15936,Bracket,1,1
3,15936,Radiator Support Lower,1,1
4,15794,Alternator Belt,1,1
...,...,...,...,...
5518,18233,Spark Plug Set Of 4,1,1
5519,18239,Rear Outer Door Handle - LH,1,1
5520,18239,Outer Door Handle - RH,1,1
5521,18238,Mirror Cap - RH,1,1


In [94]:
data_dummy.loc[len(data_dummy.index)] = [121,'Fender Liner - L/H Front', 1,1] 

In [95]:
data_dummy

,PurchaseOrderId,product_name,purchase_count,purchase_dummy
0,15791,Plug Wire,1,1
1,15790,Air Filter,1,1
2,15936,Bracket,1,1
3,15936,Radiator Support Lower,1,1
4,15794,Alternator Belt,1,1
...,...,...,...,...
5519,18239,Rear Outer Door Handle - LH,1,1
5520,18239,Outer Door Handle - RH,1,1
5521,18238,Mirror Cap - RH,1,1
5522,18238,Side Mirror Indicator - RH,1,1


In [96]:
user_id = 'PurchaseOrderId'
item_id = 'product_name'
n_rec = 3

In [97]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy),
                                            user_id=user_id, 
                                            item_id=item_id,
                                            target='purchase_dummy', similarity_type='cosine')

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 5524 observations with 2442 users and 805 items.

Data prepared in: 0.045663s

...

| 16.048ms                            | 100              | 805             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.021948s

In [98]:
def create_output(model, user_to_recommend, n_rec):
    recomendation = model.recommend(users=user_to_recommend, k=n_rec)
    return recomendation

In [101]:
user_to_recommend_item =[121]
df_output = create_output(final_model,user_to_recommend_item , n_rec)

In [102]:
print(df_output)

+-----------------+---------------------------+--------------------+------+
| PurchaseOrderId |        product_name       |       score        | rank |
+-----------------+---------------------------+--------------------+------+
|       121       |  Fender Liner - R/H Front | 0.6681531071662903 |  1   |
|       121       | Rear Bumper Bracket Right | 0.3535534143447876 |  2   |
|       121       |       Bumper - Rear       | 0.3535534143447876 |  3   |
+-----------------+---------------------------+--------------------+------+
[3 rows x 4 columns]

